In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from __future__ import division, print_function
import codecs
import os
import cPickle
from collections import OrderedDict, Counter

import numpy as np
import nltk
import h5py
from fuel.datasets import IndexableDataset, H5PYDataset


In [2]:
def extract_windows(seq, window_size):
    padded_seq = [u'_START_']*window_size + seq + [u'_END_']*window_size
    return [padded_seq[i-window_size:i+window_size+1] for i in range(window_size, window_size+len(seq))]

In [14]:
MAX_SENTENCES = 10000
WINDOW_SIZE = 2
UNKNOWN_THRESHOLD = 2
UNKNOWN_TOKEN = u'_UNK_'

tagged_sentences = nltk.corpus.brown.tagged_sents(tagset='universal')

words_by_line, tags_by_line = zip(*[zip(*sen)
                                    for sen in list(tagged_sentences)[:MAX_SENTENCES]])

# let's make a dictionary of words, and replace words which occur in < threshold sentences with u'_UNK_'
word_counts = Counter([w for line in words_by_line for w in line])
known_toks = set([k for k,v in word_counts.items() if v >= UNKNOWN_THRESHOLD])

def map_token(tok):
    if tok in known_toks:
        return tok
    return UNKNOWN_TOKEN

words_by_line = [[map_token(w) for w in line] for line in words_by_line]

word_windows, tags = zip(*[(word, tag) for word_seq, tags in zip(words_by_line, tags_by_line) 
                      for word, tag in zip(extract_windows(list(word_seq), window_size=WINDOW_SIZE), tags)])

In [18]:
# cool, lets make our train, dev, and test splits
num_instances = len(word_windows)
DEV_FRACTION = 0.2
TEST_FRACTION = 0.2

dev_size = int(np.floor(num_instances * DEV_FRACTION))
DEV_SPLIT = num_instances - dev_size
TEST_SPLIT = num_instances - int(dev_size + np.floor(num_instances * TEST_FRACTION))

# X_train, y_train = zip(*all_instances[:-TEST_SPLIT])
# X_test, y_test = zip(*all_instances[-TEST_SPLIT:-DEV_SPLIT])
# X_dev, y_dev = zip(*all_instances[-DEV_SPLIT:])

In [30]:
# split our corpus and get the set of tokens which occur in the training data
# do the same thing for tags
training_instances = word_windows[:TEST_SPLIT]
training_tags = set(tags[:TEST_SPLIT]).union(set([UNKNOWN_TOKEN]))
training_tokens = set([w for l in training_instances for w in l]).union(set([UNKNOWN_TOKEN]))
len(training_instances)

131862

In [31]:
# make a words dict, its reverse, and a tags dict and its reverse
idx2word = dict(enumerate(training_tokens))
word2idx = {v:k for k,v in idx2word.items()}

idx2tag = dict(enumerate(set([t for l in tags_by_line for t in l])))
tag2idx = {v:k for k,v in idx2tag.items()}

def map_to_index(tok, index):
    if tok in index:
        return index[tok]
    else:
        return index[UNKNOWN_TOKEN]

iwords = [[map_to_index(w, word2idx) for w in l] for l in word_windows]
itags = [map_to_index(t, tag2idx) for t in tags]

In [32]:
training_tokens

{u'writings',
 u'yellow',
 u'narcotic',
 u'four',
 u'woods',
 u'hanging',
 u'Until',
 u'aggression',
 u'marching',
 u'Foundation',
 u'eligible',
 u'Casey',
 u'Belgians',
 u'Western',
 u'immature',
 u'dependency',
 u'sinking',
 u'co-operation',
 u'regional',
 u'appropriation',
 u'Old',
 u'Heights',
 u'bringing',
 u'prize',
 u'Publique',
 u'protest',
 u'Does',
 u'succession',
 u'Paul',
 u'shows',
 u'commented',
 u"Hammarskjold's",
 u'charter',
 u'Oslo',
 u'tired',
 u'mail',
 u'tires',
 u'elegant',
 u'second',
 u'Pampa',
 u'errors',
 u'ruthless',
 u'Lucille',
 u'cooking',
 u'contributed',
 u'fingers',
 u'Seekonk',
 u'Hamilton',
 u'designing',
 u'College',
 u'increasing',
 u'Presidential',
 u'specialist',
 u'dispatched',
 u'hero',
 u'offenses',
 u'error',
 u'here',
 u'reported',
 u'affiliated',
 u'Stephanie',
 u'kids',
 u'elaborate',
 u'climbed',
 u'reports',
 u'controversy',
 u'military',
 u'Isles',
 u'rebel',
 u'criticism',
 u'golden',
 u'ground',
 u'Harvey',
 u'explained',
 u'precincts'

In [22]:
# now create the fuel dataset
# TODO: add the original data in case we need to map back

DATASET_LOCATION = 'datasets/'
DATASET_NAME = 'brown_pos_dataset.hdf5'
DATASET_PATH = os.path.join(DATASET_LOCATION, DATASET_NAME)

f = h5py.File(DATASET_PATH, mode='w')

instances = f.create_dataset('instances', (num_instances, WINDOW_SIZE*2+1), dtype='uint32')
instances[...] = np.array(iwords)

targets = f.create_dataset('targets', (num_instances, 1), dtype='uint32')
targets[...] = np.array(itags).reshape((num_instances, 1))

instances.dims[0].label = 'batch'
instances.dims[1].label = 'features'

targets.dims[0].label = 'batch'
targets.dims[1].label = 'index'

split_dict = {
    'train': {'instances': (0, TEST_SPLIT), 'targets': (0, TEST_SPLIT)},
    'test' : {'instances': (TEST_SPLIT, DEV_SPLIT), 'targets': (TEST_SPLIT, DEV_SPLIT)},
    'dev'  : {'instances': (DEV_SPLIT, num_instances), 'targets': (DEV_SPLIT, num_instances)}
}

f.attrs['split'] = H5PYDataset.create_split_array(split_dict)
f.flush()
f.close()

In [33]:
# persist the indices 

corpus_indices = {'idx2word': idx2word, 'word2idx': word2idx, 'idx2tag': idx2tag, 'tag2idx': tag2idx}

with open(os.path.join(DATASET_LOCATION, 'brown_pos_dataset.indices'), 'wb') as indices_file:
    cPickle.dump(corpus_indices, indices_file)

In [34]:
word2idx.keys()

[u'writings',
 u'yellow',
 u'narcotic',
 u'four',
 u'woods',
 u'hanging',
 u'Until',
 u'aggression',
 u'marching',
 u'Foundation',
 u'eligible',
 u'Ronald',
 u'Belgians',
 u'Western',
 u'immature',
 u'sinking',
 u'co-operation',
 u'retail',
 u'regional',
 u'appropriation',
 u'Old',
 u'Heights',
 u'bringing',
 u'Maxwell',
 u'prize',
 u'Publique',
 u'straws',
 u'Does',
 u'shouting',
 u'succession',
 u'Paul',
 u'Phyfe',
 u'commented',
 u"Hammarskjold's",
 u'charter',
 u'tired',
 u'tires',
 u'elegant',
 u'second',
 u'Pampa',
 u'Beardens',
 u'errors',
 u'ruthless',
 u'Lucille',
 u'cooking',
 u'contributed',
 u'fingers',
 u'Seekonk',
 u'Hamilton',
 u'designing',
 u'replaced',
 u'increasing',
 u'Presidential',
 u'specialist',
 u'hero',
 u'whose',
 u'error',
 u'here',
 u'reported',
 u'affiliated',
 u'Stephanie',
 u'kids',
 u'elaborate',
 u'climbed',
 u'reports',
 u'controversy',
 u'military',
 u'Isles',
 u'Ervin',
 u'criticism',
 u'golden',
 u'owed',
 u'ground',
 u'Harvey',
 u'explained',
 u'p

In [24]:
# NOW SOME QUICK TESTING

In [ ]:
train_set = H5PYDataset(DATASET_PATH, which_sets=('train',))
print(train_set.num_examples)

test_set = H5PYDataset(DATASET_PATH, which_sets=('test',))
print(test_set.num_examples)

dev_set = H5PYDataset(DATASET_PATH, which_sets=('dev',))
print(dev_set.num_examples)

in_memory_train = H5PYDataset(
    DATASET_PATH, which_sets=('train',),
    sources=['instances', 'targets'], load_in_memory=True)

# train_X, train_y = in_memory_train.data_sources

In [ ]:
from fuel.streams import DataStream
from fuel.schemes import ConstantScheme, SequentialScheme
from fuel.transformers import Batch

stream = DataStream.default_stream(in_memory_train,
                                   iteration_scheme=SequentialScheme(in_memory_train.num_examples, 500))

In [9]:
test_iter = stream.get_epoch_iterator()
for e in list(test_iter):
    print(e)

(array([[62, 62, 42, 56, 51],
       [62, 42, 56, 51, 45],
       [42, 56, 51, 45, 45],
       ..., 
       [65, 67, 49, 56, 45],
       [67, 49, 56, 45, 45],
       [62, 62, 66, 51, 64]], dtype=uint32), array([[ 4],
       [ 1],
       [ 1],
       [ 4],
       [ 1],
       [ 0],
       [ 7],
       [ 2],
       [ 1],
       [ 4],
       [ 1],
       [ 1],
       [ 1],
       [ 7],
       [ 7],
       [ 5],
       [ 0],
       [ 4],
       [11],
       [ 1],
       [ 4],
       [ 1],
       [ 7],
       [ 3],
       [ 3],
       [ 7],
       [ 2],
       [ 1],
       [ 4],
       [11],
       [ 1],
       [ 7],
       [ 2],
       [ 4],
       [ 1],
       [ 7],
       [ 0],
       [ 5],
       [ 4],
       [ 1],
       [ 7],
       [ 3],
       [ 7],
       [ 5],
       [ 4],
       [ 9],
       [ 1],
       [ 7],
       [ 7],
       [ 4],
       [ 1],
       [ 3],
       [ 7],
       [ 2],
       [ 4],
       [ 1],
       [ 5],
       [ 7],
       [ 1],
       [ 6],
       [ 1],
   

In [ ]:
# prep a corpus for POS tagging with DNNs -- use the Brown corpus from NLTK

# the baseline taggers are window-based
# data prep
# separate training, dev, and test datasets


# training pipeline
# for every training sentence, segment into windows
# 

# prediction pipeline:
# input sentence
# tokenize
# pad left and right, then extract the windows for each token (parameterized by window size)
# total vector width is feats*(window_size*2+1)

# make the fuel POS dataset from the nltk brown corpus
# TODO: write this dataset to hdf5
# TODO: split into dev, test, and train